In [52]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as tck
import pandas as pd
import sympy as sym
import warnings
import pickle as plk
from IPython.display import display, Math
from numpy.lib.scimath import sqrt as csqrt
from scipy.signal import find_peaks

import os
import imp
import sys
import importlib.util

###############################################################
# LINUX PATH

sys.path.append("/opt/lumerical/v221/api/python") #Default linux lumapi path
import lumapi
sys.path.append(os.path.dirname('/home/trainner/Documentos/Caio/GitReps/Photonics-training/Projects/Bragg Grating/SimulationFiles')) #Current directory

###############################################################
# WINDOWS PATH

# spec_win = importlib.util.spec_from_file_location('lumapi', 'C:\\Program Files\\Lumerical\\v202\\api\\python\\lumapi.py')
# lumapi = importlib.util.module_from_spec(spec_win)
# os.add_dll_directory("C:/Program Files/Lumerical/v202/api/python")
# sys.path.append(os.path.dirname('C:/Users/Caio/Documents/GitHub/Photonics-training/Projects/Bragg Grating/SimulationFiles/'))
# spec_win.loader.exec_module(lumapi)

In [53]:
bragg = lumapi.MODE(filename='BRAGGProjectEME.lms',hide=False) 

## Functions

In [54]:
# Search for function's intersections on signal levels
def signalLv(yarray,level,type):
    peak, pd = find_peaks(yarray)
    maxpeak = yarray[peak[np.argmax(yarray[peak])]]
    minpeak = yarray[np.argmin(yarray)]
    index = []

    if(type == 'linear'):
        lvpeak = (maxpeak - minpeak)*level + minpeak
        for i in range(0,np.size(yarray)-1):
            if(((maxpeak-yarray[i]) - (maxpeak-lvpeak)) * ((maxpeak-yarray[i+1]) - (maxpeak-lvpeak)) <= 0):
                index.append(i)
        yMed = (yarray[index[0]] + yarray[index[-1]])/2

    if(type == 'log10'):
        lvpeak = maxpeak + 10*np.log10(level)
        for i in range(0,np.size(yarray)-1):
            if(((maxpeak-yarray[i]) - (maxpeak-lvpeak)) * ((maxpeak-yarray[i+1]) - (maxpeak-lvpeak)) <= 0):
                index.append(i)
        yMed = 10*np.log10((10**(yarray[index[0]]/2) + 10**(yarray[index[1]]/2))/2)

    return {'index':index,'yMed':yMed}


# Creates a box
def RectInsert(X,XSPAN,Y,YSPAN,Z,ZSPAN,MATERIAL,lumerical):
    
    lumerical.switchtolayout()
    lumerical.addrect()
    lumerical.set({"x":X,"y":Y,"z":Z,
         "x span":XSPAN, "y span":YSPAN, "z span":ZSPAN,
         "material":MATERIAL
        })

## Assembly parameters

In [55]:
SilH = 450e-9 # Silicon height
SilW = 220e-9 # Silicon width

GlassH = 1e-6 # SiO2 height

ExtH = 475e-9 # Extremities height

Vp = 1e-6      # Grating Period
Vh = 50e-9     # Grating height
VpNum = 50     # Number of periods

## Assembly main model

In [56]:
bragg.switchtolayout()
bragg.deleteall()

# Substract insert
RectInsert(3*Vp, 6*Vp + 4e-6, 0, GlassH*2, -(SilW+GlassH)/2, GlassH, "SiO2 (Glass) - Palik", bragg)
bragg.set("name", "Substract")

# Waveguide insert
RectInsert(3*Vp, 6*Vp, 0, SilH, 0, SilW, "Si (Silicon) - Palik", bragg)
bragg.set("name", "Waveguide")

# Extremities insert
RectInsert(-1e-6, 2e-6, 0, ExtH, 0, SilW, "Si (Silicon) - Palik", bragg)
bragg.set("name", "W1")
RectInsert(7*Vp, 2e-6, 0, ExtH, 0, SilW, "Si (Silicon) - Palik", bragg)
bragg.set("name", "W2")

# Corrugation insert
for i in range(3):
    RectInsert(Vp/2 + 2*Vp*i, Vp, 0, ExtH+2*Vh, 0, SilW, "Si (Silicon) - Palik", bragg)
    bragg.set("name", "Teeth " + str(i+1))

# EME setting
bragg.addeme()

bragg.set({"y":0, "y span":ExtH+2*Vh, "z":0, "z span":SilW, "x min": -2e-6,
         "allow custom eigensolver settings":1 ,"display cells":1, "number of cell groups":5,
         "group spans": np.transpose([(2e-6, 2*Vp, 2*Vp, 2*Vp, 2e-6)]),
         "cells":np.transpose([1, 1, 1, 1, 1]),
         "subcell method":np.transpose([1, 1, 1, 1, 1]), # 0 = none,  1 = CVCS
         "modes":np.transpose([10, 10, 10, 10, 10])})

bragg.set({"start cell group":3, "end cell group":3, "periods":VpNum,
           "wavelength":1.5e-6,
           "mesh cells y": 200, "mesh cells z": 200})

## Main model test